# Prompt Engineering

## What is the Prompt Engineering?

`Prompt engineering` is a new discipline for developing and optimizing prompts to efficiently use language models.

There are several examples introduced at [promptingguide.ai](https://www.promptingguide.ai/introduction/examples).

In this section, we will work on recommending a movie based on 5 (or more) different movies which are closely related to the input query (i.e. same neighborhood).

In the previous section, we extracted a list of movies which are close to the input query based on ANN search with Vector Search. However, the top movies in the list still might not be the best candidates. Therefore, we will give this list of movies as a prompt and ask AI to select the best candidates based on the movie's plots.

## Why or what is the reason?

Generative AI is good at generating general information, but it does not have all the information, especially, the knowledge in a specific domain. Therefore, we will input the specialized information (in the format of a list of movies), and ask the Generative AI to select from this data. 

![full_diagram.png](images/full_diagram_system_design.png)

## Steps

1. User inputs a question. This query describes the information the user needs.

2. The Service converts the query text to a vector. The vectorization function is provided by an embedding AI model.

3. By using Vector Search, we can quickly identify the list of relevant results (which are close to the vector of the input question). However, we do not know if the closest results are the most relevant.

4. By feeding this data to Generative AI (Generate or Summarize Endpoit/Model), we can identify the most relevant results.

5. Finally, the Service returns the best candidate results to the user.

# Code

## 0. Set up

In [1]:
import cohere
from cassandra import ConsistencyLevel
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster

# get free Trial API Key at https://cohere.ai/
from cred import API_key

co = cohere.Client(API_key)

In [2]:
from cred import (ASTRA_CLIENT_ID, ASTRA_CLIENT_SECRET,
                  SECURE_CONNECT_BUNDLE_PATH)

KEYSPACE_NAME = "demo"
# 35K Movies DB
TABLE_NAME = "movies_35K_vectorized"

cloud_config = {"secure_connect_bundle": SECURE_CONNECT_BUNDLE_PATH}
auth_provider = PlainTextAuthProvider(ASTRA_CLIENT_ID, ASTRA_CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider, protocol_version=4)
session = cluster.connect()
session.execute(f"USE {KEYSPACE_NAME};")

## 1. Create a Query

In [3]:
query = "fictional beautiful animals with no human intervention"

## 2. Vectorize the Text 

In [4]:
response = co.embed(model="embed-english-light-v2.0", texts=[query])
vec = response.embeddings[0]
len(vec)

1024

## 3. Vector Search

In [5]:
method = "plot_vector_1024"
limit = 5
data = []
for row in session.execute(
    f"SELECT year, title, wiki_link, plot, plot_summary FROM {KEYSPACE_NAME}.{TABLE_NAME} "
    f" ORDER BY {method} ANN OF %s LIMIT {limit}",
    [vec]
):
    data.append(row)

for row in data:
    print(row.title, row.year)

Madagascar 3: Europe's Most Wanted 2012
Adam and Evil 1927
Fierce Creatures 1997
African Cats 2011
Doraemon: Nobita to Kiseki no Shima ~Animal Adventure~ 2012


## 4. Prompt Engineering

In [7]:
# The Generative Model might limit the length of the input data.
# Hence, we shorten the input data if exceeding the limit.
def shorten(s, n=10000):
    return ' '.join(s.split()[:n])

### Prepare the Prompt

In [9]:
prompt = f"Based on the plots of {limit} movie" + ("s" if limit > 1 else "") + f" below, suggest a movie which \"{query}\": \n"
for i, row in enumerate(data, 1):
    prompt += f"Movie {i}: title: {row.title} ({row.year}). The plot is: \n"
    #prompt += row.plot_summary  + "\n"
    prompt += shorten(row.plot, 1000) + "\n"

print(len(prompt.split()))
print(prompt)

2578
Based on the plots of 5 movies below, suggest a movie which "fictional beautiful animals with no human intervention": 
Movie 1: title: Madagascar 3: Europe's Most Wanted (2012). The plot is: 
Many days after bidding the penguins goodbye, Alex the lion suggests to his friends, Marty the zebra, Melman the giraffe and Gloria the hippopotamus and the lemurs, King Julien XIII the ring-tailed lemur, Maurice the aye-aye and Mort the mouse lemur that they should go to Monte Carlo to get the penguins and chimpanzees, Mason and Phil and fly back to New York City, which they agree to do. In Monte Carlo, the penguins and chimpanzees have been saving up their daily winnings from the casino to fly back to Africa and bring their friends home. Alex's interference to kidnap them leads to animal control, led by a vicious big-game hunter named Chantel DuBois, pursuing them around the city, with Alex and the gang barely escaping. With determination, DuBois vows to capture Alex and add his head to her

### Feed the Generate Model

The Generate Endpoints provide two Models, `command` for full functionality, and `command-nightly` for quick response.

In [16]:
response = co.generate(  
    #model='command', #
    model= 'command-nightly',   
    prompt = prompt,  
    max_tokens=126,  
    temperature=0.50)

intro_paragraph = response.generations[0].text
print(intro_paragraph)

Based on the plots of 5 movies above, suggest a movie which "fictional beautiful animals with no human intervention": 
Movie 1: title: Madagascar 3: Europe's Most Wanted (2012). The plot is: 
Many days after bidding the penguins goodbye, Alex the lion suggests to his friends, Marty the zebra, Melman the giraffe and Gloria the hippopotamus and the lemurs, King Julien XIII the ring-tailed lemur, Maurice the aye-aye and Mort the mouse lemur that they should go to Monte Carlo to get the penguins and chimpanzees, Mason and Phil and fly back


## 5. Complete the Code

In [14]:
def findMovie(text, method="plot_vector_1024", limit=2):
    response = co.embed(model="embed-english-light-v2.0", texts=[text])
    vec = response.embeddings[0]
    data = []
    for row in session.execute(
        f"SELECT year, title, wiki_link, plot, plot_summary FROM {KEYSPACE_NAME}.{TABLE_NAME} "
        f" ORDER BY {method} ANN OF %s LIMIT {limit}",
        [vec]
):
        data.append(row)

    prompt = (f"Based on the plots of {limit} movie" + ("s" if limit > 1 else "") 
              + f" below, suggest a movie which \"{query}\": \n")
    for i, row in enumerate(data, 1):
        prompt += f"Movie {i}: title: {row.title} ({row.year}). The plot is: \n"
        #prompt += row.plot_summary  + "\n"
        prompt += shorten(row.plot, 1000) + "\n"

    response = co.generate(  
        #model='command', #
        model= 'command-nightly',   
        prompt = prompt,  
        max_tokens=126,  
        temperature=0.50)

    res = response.generations[0].text

    return res

In [15]:
limit = 5
res = findMovie(query, method="plot_vector_1024", limit=limit)
print(res)

Based on the plots of 5 movies above, suggest a movie which "fictional beautiful animals with no human intervention": 
Madagascar 3: Europe's Most Wanted


In [ ]:
# Close connection to Cassandra
cluster.shutdown()